In [1]:
import sys
sys.path.append("../")

import os
from dotenv import dotenv_values
env_path = os.path.join(os.getcwd(), "../.env")
config = dotenv_values(env_path)

In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import pandas as pd
from core.utils.ql_loader import ql
from datetime import datetime, timedelta
from typing import List

from core.IRSwaps import IRSwaps, IRSwapQuery, IRSwapValue, IRSwapStructure, IRSwapStructureFunctionMap, IRSwapValueFunctionMap
from core.IRSwaptions import IRSwaptions, IRSwaptionQuery, IRSwaptionValue, IRSwaptionStructure, IRSwaptionStructureFunctionMap, IRSwaptionValueFunctionMap
from core.DataFetching.FixingsFetcher import FixingsFetcher
from core.utils.ql_utils import ql_date_to_datetime, datetime_to_ql_date

import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import matplotlib.dates as mdates
plt.style.use('ggplot')
params = {'legend.fontsize': 'x-large',
        'figure.figsize': (12, 8),
        'axes.labelsize': 'x-large',
        'axes.titlesize':'x-large',
        'xtick.labelsize':'x-large',
        'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

import nest_asyncio
nest_asyncio.apply()

Unknown QuantLib backend 'quantlib_hehehehe'. Please set QL_BACKEND to 'quantlib', 'quantlib_risks' or 'quantlib_aadc' ... Using quantlib


In [9]:
curve = "USD-SOFR-1D"
swaps_data_source = rf"CSV_{os.path.join(os.getcwd(), "..", "data", "usd_ois.csv")}"

ff = FixingsFetcher(fred_api_key=config["FRED_API_KEY"]) 
fixings = ff.get_fixings("USD-SOFR-1D")

usd_ois = IRSwaps(
    curve=curve,
    data_source=swaps_data_source,
    fixings=fixings,
    ql_interpolation_algo="log_linear",
    # pre_fetch_curves=True,
    error_verbose=True,
    max_njobs=-1,
)

In [11]:
curves = usd_ois.fetch_ql_irswap_curves(start_date=datetime(2000, 1, 1), end_date=datetime(2025, 5, 16))
# curves

BOOTSTRAPPING HISTORICAL IRSWAPS CURVE...: 100%|██████████| 6313/6313 [05:39<00:00, 18.62it/s]


In [12]:
qs = []
for f in usd_ois._DEFAULT_FWD_TENORS:
    for u in usd_ois._DEFAULT_UNDERLYING_TENORS:
        qs.append(IRSwapQuery(tenor=f"{f}x{u}" if f != "0D" else u, structure=IRSwapStructure.OUTRIGHT))

In [ ]:
ts_df = usd_ois.irswaps_timeseries_builder(
    start_date=datetime(2025, 1, 1),
    end_date=datetime(2025, 5, 10),
    # queries=[
    #     IRSwapQuery(tenor="10Yx10Y", structure=IRSwapStructure.OUTRIGHT),
    #     IRSwapQuery(tenor="1Yx10Y", structure=IRSwapStructure.OUTRIGHT),
    #     IRSwapQuery(tenor="05Yx10Y", structure=IRSwapStructure.OUTRIGHT),
    # ],
    queries=qs,
    n_jobs=-1,
)

In [16]:
ts_df

,USD-SOFR-1D 1Yx10Y OUTRIGHT RATE,USD-SOFR-1D 10Yx10Y OUTRIGHT RATE,USD-SOFR-1D 05Yx10Y OUTRIGHT RATE
Date,,,
2025-01-02 17:00:00-05:00,4.083310,4.166258,4.186833
2025-01-03 17:00:00-05:00,4.102165,4.175861,4.192377
2025-01-06 17:00:00-05:00,4.146316,4.218979,4.239025
2025-01-07 17:00:00-05:00,4.223963,4.304254,4.328716
2025-01-08 17:00:00-05:00,4.236124,4.332602,4.353596
...,...,...,...
2025-05-05 17:00:00-04:00,3.839351,4.348576,4.236885
2025-05-06 17:00:00-04:00,3.800225,4.341714,4.213016
2025-05-07 17:00:00-04:00,3.763311,4.306954,4.169355


In [21]:

datetime(2025, 1, 1) - timedelta(days=365)

datetime.datetime(2024, 1, 2, 0, 0)

In [5]:
eur_ois = IRSwaps(
    curve="EUR-ESTR",
    data_source="CME",
    # fixings=ff.get_fixings("EUR-ESTR"),
    ql_interpolation_algo="log_linear",
    pre_fetch_curves=True,
    error_verbose=True,
    max_njobs=-1,
)

In [28]:
# eur_ois.irswaps_term_structure_plotter(bdates=[datetime(2025, 3, 31), datetime(2025, 5, 9)], fwd_tenors=["0D", "1Y"], use_plotly=True)

In [ ]:
PAR_TENORS = [
    "1D",
    "1W",
    "2W",
    "3W",
    "1M",
    "2M",
    "3M",
    "4M",
    "5M",
    "6M",
    "7M",
    "8M",
    "9M",
    "10M",
    "11M",
    "12M",
    "15M",
    "18M",
    "21M",
    "2Y",
    "3Y",
    "4Y",
    "5Y",
    "6Y",
    "7Y",
    "8Y",
    "9Y",
    "10Y",
    "12Y",
    "15Y",
    "20Y",
    "25Y",
    "30Y",
    "40Y",
    "50Y",
]

ts_df = eur_ois.irswaps_timeseries_builder(
	start_date=datetime(2020, 1, 1),
	end_date=datetime(2025, 5, 16),
	queries=[(IRSwapQuery(tenor=t), t) for t in PAR_TENORS],
	n_jobs=-1,
    ignore_cache=True
)

Fetching Curve Reports:   0%|          | 0/1027 [00:00<?, ?it/s]

In [16]:
ts_df[PAR_TENORS].to_csv(r"C:\Users\chris\Project Bond King\rvcore\data\eur_ois.csv")